In [8]:
import numpy as np
import scipy.linalg as la

In [42]:
### Helper Functions ###

# Returns the kronecker product of many operators
def multikron(ops):
    if len(ops) >=3:
        return np.kron(ops[0], multikron(ops[1:]))
    else:
        return np.kron(ops[0], ops[1])

# Arbitrary CNOT matrix (supports nonadjacent qubits)
## c = control, t = target, n = number of qubits
def arb_cx(c, t, n):
    ops0 = [id] * n
    ops0[c-1] = np.array([[1,0],[0,0]])
    
    ops1 = [id] * n
    ops1[c-1] = np.array([[0,0],[0,1]])
    ops1[t-1] = np.array([[0,1],[1,0]])

    return multikron(ops0) + multikron(ops1)

# Arbitrary Controlled-V (supports nonadjacent qubits)
def arb_cv(ctrl, t, n, a, b, c, d):
    opsDA = [id] * n
    opsIB = [id] * n
    opsIC = [id] * n

    opsDA[ctrl-1] = d
    opsDA[t - 1] = a

    opsIB[t-1] = b
    opsIC[t-1] = c

    return multikron(opsDA) @ arb_cx(ctrl,t,n) @ multikron(opsIB) @ arb_cx(ctrl,t,n) @ multikron(opsIC)



# Returns parameterized unitary as given in the task
def U(t, p, l):
    return np.array([[np.cos(t/2), -np.exp(1j*l)*np.sin(t/2)],
                     [np.exp(1j*p)*np.sin(t/2), np.exp(1j*(p+l))*np.cos(t/2)]])

In [10]:
### Useful Operators ###
id = np.eye(2)
x = np.array([[0,1],[1,0]])
y = np.array([[0,-1j],[1j,0]])
z = np.array([[1,0],[0,-1]])
cx = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [18]:
A = la.expm((-1j/2)*(-3*np.pi/2)*z) @ la.expm((-1j/4)*(-np.pi/2)*y)
B = la.expm((1j/4)*(-np.pi/2)*y) @ la.expm((1j/4*(-2*np.pi)*z))
C = la.expm((1j/4)*(-np.pi)*z)
D = np.array([[1,0],[0,np.exp(1j*(-3*np.pi/4))]])

In [38]:
## D has theta = 0, phi = 0 , lambda = -3pi/4
## C has theta, phi = 0 and lambda = pi/2 and a global phase exp(-i*pi/4) 
## B has a global phase exp(-i*pi/2) and theta = pi/4, phi = 0, lambda = pi
## A has global phase exp(3i*pi/4), theta = pi/4, phi = 3pi/2, lamda = pi

## Conjugates are trivially constructed by with the same theta, phi' = -lambda - pi, lambda' = -phi - pi

a = np.exp(3*1j*np.pi/4) * U(np.pi/4, 3*np.pi/2, np.pi)
a_dag = np.exp(-3*1j*np.pi/4) * U(np.pi/4, -np.pi - np.pi, -3*np.pi/2 - np.pi)
print(A - a)
print(a@a_dag)
print('\n')

b = np.exp(-1j*np.pi/2) * U(np.pi/4, 0, np.pi)
b_dag = np.exp(1j*np.pi/2) * U(np.pi/4, -np.pi - np.pi, -np.pi)
print(B - b)
print(b@b_dag)
print('\n')

c = np.exp(-1j*np.pi/4) * U(0, 0, np.pi/2)
c_dag = np.exp(1j*np.pi/4) * U(0, -np.pi/2 - np.pi, -np.pi)
print(C - c)
print(c@c_dag)
print('\n')

d = U(0, 0, -3*np.pi/4)
d_dag = U(0, 3*np.pi/4 - np.pi, -np.pi)
print(D - d)
print(d@d_dag)
print('\n')

[[ 0.00000000e+00+0.00000000e+00j -5.55111512e-17-5.55111512e-17j]
 [-1.11022302e-16+1.11022302e-16j  3.33066907e-16-3.33066907e-16j]]
[[ 1.00000000e+00+0.00000000e+00j  4.59925572e-17-2.39291222e-17j]
 [-1.40922799e-17+2.39291222e-17j  1.00000000e+00+1.11022302e-16j]]


[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.00000000e+00+1.79345371e-17j 2.16349973e-17-4.32978028e-17j]
 [2.16349973e-17-4.32978028e-17j 1.00000000e+00+2.26994823e-16j]]


[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.00000000e+00j 0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.+3.33066907e-16j]]


[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.00000000e+00j 0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.+2.22044605e-16j]]




In [45]:
CV = np.kron(d,a) @ cx @ np.kron(id,b) @ cx @ np.kron(id,c)
CV_13 = arb_cv(1,3,3,a,b,c,d)

CV_dag = np.kron(id,c_dag) @ cx @ np.kron(id,b_dag) @ cx @ np.kron(d_dag,a_dag)

CCX = CV_13 @ np.kron(cx, id) @ np.kron(id, CV_dag) @ np.kron(cx, id) @ np.kron(id, CV)

print(CCX)

[[ 1.00000000e+00-2.55184613e-16j -3.24991196e-18+3.62071836e-17j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-5.33759432e-17-2.45777490e-17j  1.00000000e+00+3.29744180e-16j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   1.00000000e+00+2.77555756e-16j -1.38777878e-16+5.55111512e-17j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   2.77555756e-17+1.11022302e-16j  1.00000000e+00+3.60822483e-16j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00